In [1]:
import warnings

warnings.filterwarnings("ignore")

import time
import random
import numpy as np
import pandas as pd
from tinysmpc.tinysmpc import VirtualMachine, PrivateScalar
from tinysmpc.tinysmpc.fixed_point import float_point, fixed_point

import torch
import flwr as fl
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import random_split, TensorDataset, DataLoader

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}")

Training on cpu using PyTorch 1.13.1+cpu and Flower 1.1.0


In [6]:
N = 4
Q = 2657003489534545107915232808830590043

In [7]:
def additive_share(secret, N):
    shares = [random.randrange(Q) for _ in range(N - 1)]
    shares += [(secret - sum(shares)) % Q]
    return shares


def additive_reconstruct(shares):
    return sum(shares) % Q


def share_tensor(tensor, n):
    random_values = [random.randrange(Q) for _ in range(np.prod((tensor.shape + (n - 1,))))]
    random_shares = np.array(random_values).reshape(n - 1, tensor.shape[0], tensor.shape[1])
    n_shares = np.concatenate([random_shares, [(tensor - random_shares.sum(axis=0)) % Q]])
    return n_shares


def additive_reconstruct_tensor(shares):
    secrets = shares.sum(axis=0) % Q
    return secrets

# Fixed Point Conversion Testing
### Single Values

In [8]:
test_float_point = 1.2345

test_fixed_point = fixed_point(test_float_point)

test_fixed_point_shares = additive_share(test_fixed_point, 3)

test_fixed_point_recon = additive_reconstruct(test_fixed_point_shares)

test_reconstructed_float_point = float_point(test_fixed_point)

print(f"Test Float Point: {test_float_point}")
print(f"Test Fixed Point (Converted): {test_fixed_point}")
print(f"Test Fixed Point Shares: {test_fixed_point_shares}")
print(f"Test Fixed Point Additive Reconstructed: {test_fixed_point_recon}")
print(f"Test Float Point Reconstructed: {test_reconstructed_float_point}")

Test Float Point: 1.2345
Test Fixed Point (Converted): 123450000
Test Fixed Point Shares: [283714451870265387167790872036476803, 1549885977164866690853642226773371265, 823403060499413029893799710144191975]
Test Fixed Point Additive Reconstructed: 123450000
Test Float Point Reconstructed: 1.2345


### Tensor Values

In [9]:
fixedPoint = np.vectorize(fixed_point)
floatPoint = np.vectorize(float_point)

float_point_secret = np.random.rand(4, 3)

fixed_point_secret = fixedPoint(float_point_secret)
float_point_secret_reconstructed = floatPoint(fixed_point_secret)

print(f"Float Point Secret\n{float_point_secret}\n")
print(f"Fixed Point Secret (Converted)\n{fixed_point_secret}\n")
print(f"Float Point Secret Reconstructed\n{float_point_secret_reconstructed}\n")

Float Point Secret
[[0.65505991 0.29824622 0.95773975]
 [0.79589814 0.20183362 0.12633212]
 [0.74974386 0.14065554 0.20362259]
 [0.13407458 0.46753325 0.51083551]]

Fixed Point Secret (Converted)
[[65505991 29824622 95773975]
 [79589813 20183361 12633212]
 [74974385 14065553 20362259]
 [13407458 46753324 51083550]]

Float Point Secret Reconstructed
[[0.65505991 0.29824622 0.95773975]
 [0.79589813 0.20183361 0.12633212]
 [0.74974385 0.14065553 0.20362259]
 [0.13407458 0.46753324 0.5108355 ]]



### Integer Tensor Additive Sharing Test

In [10]:
secret_int = np.random.randint(1, 20, (4, 3))
print(f"Secret\n{secret_int}\n")

test_shares = share_tensor(secret_int, N)
# print(f"Secret Shares: \n{test_shares}\n")

secret_reconstructed = additive_reconstruct_tensor(test_shares)

print(f"Reconstructed Secret\n{secret_reconstructed}\n")

Secret
[[ 9 18 19]
 [12  3  1]
 [16 11  8]
 [ 5 19 11]]

Reconstructed Secret
[[9 18 19]
 [12 3 1]
 [16 11 8]
 [5 19 11]]



### Float Tensor Additive Sharing Test

In [11]:
secret_int = np.random.rand(4, 3)
print(f"Secret\n{secret_int}\n")

fixed_point_secret = fixedPoint(float_point_secret)
print(f"Fixed Point Secret\n{fixed_point_secret}\n")

fixed_point_test_shares = share_tensor(fixed_point_secret, N)
fixed_point_secret_reconstructed = additive_reconstruct_tensor(fixed_point_test_shares)

print(f"Reconstructed Fixed Point Secret\n{fixed_point_secret_reconstructed}\n")
print(f"Reconstructed Float Point Secret\n{floatPoint(fixed_point_secret_reconstructed)}\n")

Secret
[[0.77737174 0.33138294 0.60599308]
 [0.35832953 0.2501239  0.81509734]
 [0.20232609 0.26429611 0.30046175]
 [0.38385028 0.81012827 0.0835832 ]]

Fixed Point Secret
[[65505991 29824622 95773975]
 [79589813 20183361 12633212]
 [74974385 14065553 20362259]
 [13407458 46753324 51083550]]

Reconstructed Fixed Point Secret
[[65505991 29824622 95773975]
 [79589813 20183361 12633212]
 [74974385 14065553 20362259]
 [13407458 46753324 51083550]]

Reconstructed Float Point Secret
[[0.65505991 0.29824622 0.95773975]
 [0.79589813 0.20183361 0.12633212]
 [0.74974385 0.14065553 0.20362259]
 [0.13407458 0.46753324 0.5108355 ]]



In [12]:
parameters = [
    ["kofi", "ama", "kojo"],
    [40000, 15000, -20000],
]

"""
Secret sharing process
"""
#
aggregator = VirtualMachine("aggregator")

# Edge Devices
nodes = [VirtualMachine(node_id) for node_id in parameters[0]]

# Tensors of edge devices
node_values = [PrivateScalar(tensor, node) for tensor, node in zip(parameters[1], nodes)]

# Nodes with their local shares
exchanged_shares = []

for value in node_values:
    exchanged_shares.append(value.share(nodes))

for node in exchanged_shares:
    print(node)

shared_sum = sum(exchanged_shares)
shared_sum.reconstruct(aggregator)

SharedScalar
 - Share(-8901149815987637370, 'kofi', Q=None)
 - Share(7242444370969791757, 'ama', Q=None)
 - Share(1658705445017885613, 'kojo', Q=None)
SharedScalar
 - Share(-8079527532602339583, 'kofi', Q=None)
 - Share(-3732971907603757191, 'ama', Q=None)
 - Share(-6634244633503439842, 'kojo', Q=None)
SharedScalar
 - Share(-9111602508798889657, 'kofi', Q=None)
 - Share(4878177505536876685, 'ama', Q=None)
 - Share(4233425003261992972, 'kojo', Q=None)


PrivateScalar(35000, 'aggregator')

In [18]:
tensor_1 = fixedPoint(np.random.rand(4, 3))
tensor_2 = fixedPoint(np.random.rand(4, 3))
tensor_3 = fixedPoint(np.random.rand(4, 3))

fl_node_weights = [
    ["kofi", "ama", "kojo"],
    [tensor_1, tensor_2, tensor_3],
]

"""
Secret sharing process
"""
#
fl_server = VirtualMachine("fl_server")

# Edge Devices
fl_nodes = [VirtualMachine(node_id) for node_id in fl_node_weights[0]]

# Tensors of edge devices
fl_node_values = [PrivateScalar(tensor, node) for tensor, node in zip(fl_node_weights[1], fl_nodes)]

# Nodes with their local shares
fl_exchanged_shares = []

for value in fl_node_values:
    fl_exchanged_shares.append(value.share_tensor(fl_nodes, Q))

shared_sum = sum(fl_exchanged_shares)
shared_sum.reconstruct(fl_server)

original_sum = np.array(fl_node_weights[1]).sum(axis=0)

print(f"Original Sum\n{original_sum}\n")
print(f"Shared Sum\n{fl_server.objects[-1]}\n")

Original Sum
[[152583182 109968855 137267852]
 [182664572  74534249 117207282]
 [127671913 203622001 132459203]
 [166374039 167008605  91701328]]

Shared Sum
PrivateScalar([[152583182 109968855 137267852]
 [182664572 74534249 117207282]
 [127671913 203622001 132459203]
 [166374039 167008605 91701328]], 'fl_server')



## Pytorch Integration
### Simple ANN in pytorch

In [15]:
NUM_CLIENTS = 5
BATCH_SIZE = 10


def load_datasets():
    dataset = pd.read_csv("./iris.csv")
    dataset.columns = ["sepal length (cm)",
                       "sepal width (cm)",
                       "petal length (cm)",
                       "petal width (cm)",
                       "species"]

    mappings = {
        "Iris-setosa": 0,
        "Iris-versicolor": 1,
        "Iris-virginica": 2
    }
    dataset["species"] = dataset["species"].apply(lambda x: mappings[x])

    X = dataset.drop("species", axis=1).values
    y = dataset["species"].values

    inputs = torch.tensor(X, dtype=torch.float32)
    targets = torch.tensor(y, dtype=torch.float32)

    ds = TensorDataset(inputs, targets)

    train_size = 100
    test_size = len(ds) - train_size

    train_ds, test_ds = random_split(ds, [train_size, test_size], torch.Generator().manual_seed(42))

    # Split training set into 10 partitions to simulate the individual dataset
    partition_size = len(train_ds) // NUM_CLIENTS
    lengths = [partition_size] * NUM_CLIENTS
    datasets = random_split(train_ds, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))

    testloader = DataLoader(test_ds, batch_size=BATCH_SIZE)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets()

In [16]:
input_dim = 4
hidden_dim = 25
output_dim = 3


class Model(nn.Module):
    def __init__(self, input_features, hidden_layer, output_features):
        super().__init__()
        self.fc1 = nn.Linear(input_features, hidden_layer)
        self.out = nn.Linear(hidden_layer, output_features)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.out(x)
        return x

In [21]:
def train(net, trainloader, epochs: int, verbose=False):
    """Train the network on the training set."""

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

    net.train()
    training_start_time = time.time()

    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for X_train, y_train in trainloader:
            optimizer.zero_grad()
            y_pred = net.forward(X_train)
            loss = criterion(y_pred, y_train)
            loss.backward()
            optimizer.step()

            # Metrics
            epoch_loss += loss
            total += y_train.size(0)
            correct += (torch.max(y_pred.data, 1)[1] == y_train).sum().item()

        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        if verbose:
            print(
                f"Epoch {epoch + 1}: train loss:  {epoch_loss}, "
                f"accuracy: {epoch_acc}, "
                f"time taken: {time.time() - training_start_time}"
            )


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for X_train, y_train in testloader:
            outputs = net(X_train)
            loss += criterion(outputs, y_train).item()
            _, predicted = torch.max(outputs.data, 1)
            total += y_train.size(0)
            correct += (predicted == y_train).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [22]:
trainloader = trainloaders[0]
valloader = valloaders[0]
net = Model(input_dim, hidden_dim, output_dim).to(DEVICE)

train(net, trainloader, 5, True)

loss, accuracy = test(net, testloader)
print(f"Final test set performance:\n\tloss {loss}\n\taccuracy {accuracy}")

RuntimeError: expected scalar type Long but found Float

In [31]:
model = Model(input_dim, hidden_dim, output_dim)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [32]:
epochs = 100
losses = []

for i in range(epochs):
    y_pred = model.forward(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss)
    print(f'epoch: {i:2}  loss: {loss.item():10.8f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

epoch:  0  loss: 1.50877154
epoch:  1  loss: 1.31337249
epoch:  2  loss: 1.19190168
epoch:  3  loss: 1.12893331
epoch:  4  loss: 1.11746204
epoch:  5  loss: 1.12976313
epoch:  6  loss: 1.13385344
epoch:  7  loss: 1.12027764
epoch:  8  loss: 1.09459150
epoch:  9  loss: 1.06500697
epoch: 10  loss: 1.03642476
epoch: 11  loss: 1.00935745
epoch: 12  loss: 0.98309314
epoch: 13  loss: 0.95777440
epoch: 14  loss: 0.93439484
epoch: 15  loss: 0.91399240
epoch: 16  loss: 0.89615339
epoch: 17  loss: 0.87954271
epoch: 18  loss: 0.86441445
epoch: 19  loss: 0.84770775
epoch: 20  loss: 0.82643974
epoch: 21  loss: 0.80183363
epoch: 22  loss: 0.77610856
epoch: 23  loss: 0.75138932
epoch: 24  loss: 0.72907358
epoch: 25  loss: 0.70935827
epoch: 26  loss: 0.69152653
epoch: 27  loss: 0.67423224
epoch: 28  loss: 0.65653789
epoch: 29  loss: 0.63821149
epoch: 30  loss: 0.61967319
epoch: 31  loss: 0.60161477
epoch: 32  loss: 0.58465350
epoch: 33  loss: 0.56916916
epoch: 34  loss: 0.55497640
epoch: 35  loss: 0.5